In [5]:
!pip install scipy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [6]:
import pandas as pd
import numpy as np
import time

In [7]:
df = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Github_Repos/Twitter-Sentiment-Analysis/data/sampled/training.1600000.processed.noemoticon.csv", encoding = "latin-1", low_memory=False)

In [8]:
df.head()

,ids,date,user,text,target
0,2265878782,Sun Jun 21 07:29:15 PDT 2009,MeeJong,This morning my daughter asked me if I hate my...,0
1,2205565064,Wed Jun 17 05:00:35 PDT 2009,kathysyahrizal,"I forgot to charge my bb, zzzzz only 35% left",0
2,2002872654,Tue Jun 02 05:22:07 PDT 2009,kgautam,my MBP battery is fluctuating between dead to ...,0
3,1835496893,Mon May 18 06:07:33 PDT 2009,thejanice,i really wish my landlord would call me back. ...,0
4,1573596472,Tue Apr 21 01:55:46 PDT 2009,lucyxechelon,mean blog people,0


In [9]:
df.shape

(50000, 5)

In [10]:
import gensim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [11]:
# use pretrained google word2vec model
model_word2vec = gensim.models.KeyedVectors.load_word2vec_format("/Users/jeremyhudsonchan/Dropbox/Files/Github_Repos/Twitter-Sentiment-Analysis/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [12]:
# apply model_word2vec to each text in use_df
def word2vec(text):
    text = text.split()
    # use keyedvector's .get_vector() method to get the vector for each word in text
    # if word is not in model_word2vec, return a vector of 0s
    text = [model_word2vec.get_vector(word) if word in model_word2vec else np.zeros(300) for word in text]
    return text

# apply word2vec to each text in use_df
df['text'] = df['text'].apply(word2vec)

In [13]:
print(df['text'].head(1))

0    [[-0.2890625, 0.19921875, 0.16015625, 0.025268...
Name: text, dtype: object


In [14]:
# sum all the vectors in each text
def sum_vectors(text):
    text = np.array(text)
    text = np.sum(text, axis=0)
    return text

# apply sum_vectors to each text in use_df
df['text'] = df['text'].apply(sum_vectors)

In [15]:
print(df['text'].head(2))

0    [0.6248779296875, 0.260833740234375, 1.0648345...
1    [0.875732421875, 0.374603271484375, 0.68841552...
Name: text, dtype: object


In [16]:
print(len(df['text']))

50000


In [17]:
# check if each text has 300 elements
for i in range(len(df['text'])):
    if len(df['text'].iloc[i]) != 300:
        print(i)

In [18]:
# make use_df['text'] to a pandas dataframe with 300 columns, keeping the indexs
temp_df = pd.DataFrame(df['text'].tolist(), index=df.index)
# add target column to temp_df
temp_df['target'] = df['target']

In [19]:
temp_df['target'].head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [20]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(temp_df.drop('target', axis=1), temp_df['target'], test_size=0.2, random_state=5)

In [21]:
# logistic regression, with train_x being a list of lists
from sklearn.linear_model import LogisticRegression
def logistic_regression(train_x, train_y):
  classifier = LogisticRegression(max_iter=100000)
  classifier.fit(train_x, train_y)
  return classifier

# svm
from sklearn.svm import SVC
def svm(train_x, train_y):
    # take a subset of train_x and train_y
    classifier = SVC(max_iter=100000, C=1.0)
    classifier.fit(train_x, train_y)
    return classifier

# knn
from sklearn.neighbors import KNeighborsClassifier
def knn(train_x, train_y):
  # take a subset of train_x and train_y
  classifier = KNeighborsClassifier()
  classifier.fit(train_x, train_y)
  return classifier

# decision tree
from sklearn import tree
def decision_tree(train_x, train_y):
  classifier = tree.DecisionTreeClassifier()
  classifier.fit(train_x, train_y)
  return classifier

In [22]:
# run logistic regression
import time
start_time = time.time()
classifier = logistic_regression(X_train, y_train)
log_predictions = classifier.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

0.7192
--- 2.091555118560791 seconds ---


In [ ]:
# evaluate logistic regression
print(accuracy_score(y_test, log_predictions))
print(classification_report(y_test, log_predictions))
print(confusion_matrix(y_test, log_predictions))

In [ ]:
# plot roc curve
log_fpr, log_tpr, log_thresold = roc_curve(y_test, log_predictions, pos_label=4)
plt.plot(log_fpr, log_tpr, label="Logistic Regression")
plt.plot([0,1], [0,1], label="Baseline")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()
# print auc score
print(roc_auc_score(y_test, log_predictions))

In [23]:
# run svm
start_time = time.time()
classifier = svm(X_train, y_train)
predictions = classifier.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

0.7352
--- 535.0357539653778 seconds ---


In [ ]:
# evaluate svm
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
# plot roc curve
svm_fpr, svm_tpr, svm_thresold = roc_curve(y_test, predictions, pos_label=4)
plt.plot(svm_fpr, svm_tpr, label="SVM")
plt.plot([0,1], [0,1], label="Baseline")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()
# print auc score
print(roc_auc_score(y_test, predictions))

In [24]:
# run knn
start_time = time.time()
classifier = knn(X_train, y_train)
predictions = classifier.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

0.6582
--- 1.8101298809051514 seconds ---


In [ ]:
# evaluate knn
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
# plot roc curve
knn_fpr, knn_tpr, knn_thresold = roc_curve(y_test, predictions, pos_label=4)
plt.plot(knn_fpr, knn_tpr, label="KNN")
plt.plot([0,1], [0,1], label="Baseline")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()
# print auc score
print(roc_auc_score(y_test, predictions))

In [25]:
# run decision tree
start_time = time.time()
classifier = decision_tree(X_train, y_train)
predictions = classifier.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

0.5964
--- 16.029811143875122 seconds ---


In [ ]:
# evaluate decision tree
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
# plot roc curve
dt_fpr, dt_tpr, dt_thresold = roc_curve(y_test, predictions, pos_label=4)
plt.plot(dt_fpr, dt_tpr, label="Decision Tree")
plt.plot([0,1], [0,1], label="Baseline")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()
# print auc score
print(roc_auc_score(y_test, predictions))